In [1]:
! pip install -q tensorflow-hub
! pip install -q seaborn
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from preprocess import preprocessing
from preparation import preparation

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [4]:
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

# Reduce logging output.
logging.set_verbosity(logging.ERROR)

message_embeddings = embed(messages)


for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
  print("Message: {}".format(messages[i]))
  print("Embedding size: {}".format(len(message_embedding)))
  message_embedding_snippet = ", ".join(
      (str(x) for x in message_embedding[:3]))
  print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: Elephant
Embedding size: 512
Embedding: [0.008344464004039764, 0.00048085112939588726, 0.06595247983932495, ...]

Message: I am a sentence for which I would like to get its embedding.
Embedding size: 512
Embedding: [0.050808604806661606, -0.016524311155080795, 0.015737788751721382, ...]

Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding: [-0.02833266742527485, -0.0558621808886528, -0.012941486202180386, ...]



In [ ]:
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  sns.set(font_scale=1.2)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")

def run_and_plot(messages_):
  message_embeddings_ = embed(messages_)
  plot_similarity(messages_, message_embeddings_, 90)

In [14]:
import os
import sklearn.metrics.pairwise as pairwise

file_path_originals = "original_files"
file_path_suspicious = "suspicious_files"

# Preprocessing original texts
original_texts = [file for file in os.listdir(file_path_originals) if os.path.isfile(os.path.join(file_path_originals, file))]
processed_original_texts = []
for original_text in original_texts:
    processed_original_texts.append(preprocessing(file_path_originals + "/" + original_text))

print("Starting plagiarism detection...")
print("\n")
# Preprocessing suspicious texts
suspicious_texts = [file for file in os.listdir(file_path_suspicious) if os.path.isfile(os.path.join(file_path_suspicious, file))]
processed_suspicious_texts = []
for suspicious_text in suspicious_texts:
    processed_suspicious_texts.append(preprocessing(file_path_suspicious + "/" + suspicious_text))


embeddings_sus = []
embeddings_ori = []


# for i in range(len(processed_suspicious_texts)):
embeddings_sus.append(embed(processed_suspicious_texts))
embeddings_ori.append(embed(processed_original_texts))\

# reshape to only have np array
embeddings_sus = np.array(embeddings_sus)
embeddings_ori = np.array(embeddings_ori)

# reshape to only have np array
embeddings_sus = np.reshape(embeddings_sus, (1,512))
embeddings_ori = np.reshape(embeddings_ori, (1,512))


uni_result = pairwise.cosine_similarity(embeddings_sus)[0,1]
tri_result = pairwise.cosine_similarity(embeddings_sus)[0,1]
print("Similarity between suspicious text " + str(i) + " and original text " + str(i) + " is: " + str(uni_result))
print("Similarity between suspicious text " + str(i) + " and original text " + str(i) + " is: " + str(tri_result))

print(embeddings_ori, embeddings_sus)







Starting plagiarism detection...


(1, 95, 512) (1, 15, 512)


ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.